
# Predicting Air Pollution Levels using Linear Models 

This notebook demonstrates the steps to use a Ridge Regression model to predict air pollution levels.

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.metrics import confusion_matrix
from sklearn.impute import KNNImputer



# Redo the analysis with timeseries split 

In [11]:
full_data = pd.read_csv("prepped_data/Full_data.csv") 
full_data.columns

Index(['Hour', 'date', 'NO_ugm3', 'NO2_ugm3', 'O3_ugm3', 'CO_mgm3', 'CO2_mgm3',
       'PM25_ugm3', 'SiteID', 'Lat', 'Long', 'day_of_week', 'avgtempC',
       'maxtempC', 'mintempC', 'sunHour', 'uvIndex', 'humidity',
       'winddirDegree', 'windspeedKmph', 'cloudcover', 'precipMM', 'pressure',
       'DCC-AQ1-co', 'DCC-AQ1-no', 'DCC-AQ10-no', 'DCC-AQ13-no', 'DCC-AQ5-no',
       'DCC-AQ6-no', 'DCC-AQ1-no2', 'DCC-AQ10-no2', 'DCC-AQ13-no2',
       'DCC-AQ22-no2', 'DCC-AQ5-no2', 'DCC-AQ6-no2', 'DCC-AQ69-no2',
       'DCC-AQ22-o3', 'DCC-AQ69-o3', 'DCC-AQ10-pm1', 'DCC-AQ13-pm1',
       'DCC-AQ2-pm1', 'DCC-AQ3-pm1', 'DCC-AQ4-pm1', 'DCC-AQ5-pm1',
       'DCC-AQ52-pm1', 'DCC-AQ6-pm1', 'TNO2161-pm1', 'TNO2162-pm1',
       'TNO4435-pm1', 'TNT1088-pm1', 'DCC-AQ10-pm10', 'DCC-AQ13-pm10',
       'DCC-AQ2-pm10', 'DCC-AQ22-pm10', 'DCC-AQ3-pm10', 'DCC-AQ4-pm10',
       'DCC-AQ5-pm10', 'DCC-AQ52-pm10', 'DCC-AQ6-pm10', 'TNO2161-pm10',
       'TNO2162-pm10', 'TNO4435-pm10', 'TNT1088-pm10', 'DCC-AQ10-pm2_

In [12]:
df_test = full_data[full_data['date'] >= '2022-05-01']
df_train = full_data[full_data['date'] < '2022-05-01']

In [13]:
# #Split train into X and Y
Xtrain = df_train.iloc[:, 8:].values
ytrain = df_train["PM25_ugm3"].values

# #Split test into X and Y
Xtest = df_test.iloc[:, 8:].values
ytest = df_test["PM25_ugm3"].values


In [14]:
# Assuming you know the names of the categorical columns
categorical_columns = ['SiteID', 'day_of_week']  # List of categorical column names

# Convert arrays back to DataFrame for easier manipulation
Xtrain_df = pd.DataFrame(Xtrain, columns=df_train.columns[8:])
Xtest_df = pd.DataFrame(Xtest, columns=df_test.columns[8:])

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit the encoder on the training data
encoder.fit(Xtrain_df[categorical_columns])

# Transform both training and test data
Xtrain_encoded = encoder.transform(Xtrain_df[categorical_columns])
Xtest_encoded = encoder.transform(Xtest_df[categorical_columns])

# Create DataFrames from the encoded arrays, include column names for easier merging
encoded_columns = encoder.get_feature_names_out(categorical_columns)
Xtrain_encoded_df = pd.DataFrame(Xtrain_encoded, columns=encoded_columns)
Xtest_encoded_df = pd.DataFrame(Xtest_encoded, columns=encoded_columns)

# Drop the original categorical columns and concat the new encoded columns
Xtrain_final = pd.concat([Xtrain_df.drop(categorical_columns, axis=1), Xtrain_encoded_df], axis=1)
Xtest_final = pd.concat([Xtest_df.drop(categorical_columns, axis=1), Xtest_encoded_df], axis=1)


c:\Users\Giulia Maria\miniconda3\envs\ML\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
np.random.seed(0)   
# imputing 
imputer = SimpleImputer(strategy="mean")
Xtrain_1 = imputer.fit_transform(Xtrain_final)  # Impute
Xtest_1 = imputer.transform(Xtest_final)  # Impute

# Convert back to DataFrame
Xtrain_1 = pd.DataFrame(Xtrain_1, columns=Xtrain_final.columns)
Xtest_1 = pd.DataFrame(Xtest_1, columns=Xtest_final.columns)

# scaling 
scaler = StandardScaler()
Xtrain_1 = scaler.fit_transform(Xtrain_1)  # Scale
Xtest_1 = scaler.transform(Xtest_1)  # Scale

# Convert back to DataFrame
Xtrain_1 = pd.DataFrame(Xtrain_1, columns=Xtrain_final.columns)
Xtest_1 = pd.DataFrame(Xtest_1, columns=Xtest_final.columns)

In [32]:
np.random.seed(0)   

imputer = KNNImputer(n_neighbors=5)
Xtrain_KNN = imputer.fit_transform(Xtrain_final)
Xtest_KNN = imputer.transform(Xtest_final)

# Convert back to DataFrame
Xtrain_2 = pd.DataFrame(Xtrain_KNN, columns=Xtrain_final.columns)
Xtest_2 = pd.DataFrame(Xtest_KNN, columns=Xtest_final.columns)

Xtrain = scaler.fit_transform(Xtrain_2)
Xtest = scaler.transform(Xtest_2)

# Convert back to DataFrame
Xtrain_KNN = pd.DataFrame(Xtrain, columns=Xtrain_final.columns)
Xtest_KNN = pd.DataFrame(Xtest, columns=Xtest_final.columns)

In [39]:
print(Xtrain_KNN.shape) 
print(Xtest_KNN.shape)  

print(ytrain.shape)
print(ytest.shape)

print(Xtrain_1.shape)
print(Xtest_1.shape)

print(ytrain.shape)
print(ytest.shape)

(34923, 901)
(8272, 901)
(34923,)
(8272,)
(34923, 901)
(8272, 901)
(34923,)
(8272,)


In [44]:
Xtrain_KNN.to_csv('prepped_data/Xtrain_KNN_encoded.csv', index=False)
Xtest_KNN.to_csv('prepped_data/Xtest_KNN_encoded.csv', index=False)

In [45]:
# killian thing
Xtrain_KNN_Killian = pd.read_csv("prepped_data/Xtrain_KNN.csv")
Xtest_KNN_Killian = pd.read_csv("prepped_data/Xtest_KNN.csv")   


In [46]:
# mine

Xtrain_KNN_encoded = pd.read_csv("prepped_data/Xtrain_KNN_encoded.csv")
Xtest_KNN_encoded = pd.read_csv("prepped_data/Xtest_KNN_encoded.csv")   


        Lat      Long  avgtempC  maxtempC  mintempC   sunHour   uvIndex  \
0 -1.128775 -1.358738 -1.330213 -1.137422 -1.953354  0.900165 -0.207077   
1 -0.759257 -1.584956 -1.330213 -1.137422 -1.953354  0.900165 -0.207077   
2 -1.188387 -1.714836 -1.330213 -1.137422 -1.953354  0.900165 -0.207077   
3 -1.048941 -2.050665 -1.330213 -1.137422 -1.953354  0.900165 -0.207077   
4 -2.137602 -1.135172 -1.330213 -1.137422 -1.953354  0.900165 -0.207077   

   humidity  winddirDegree  windspeedKmph  ...  SiteID_964.0  SiteID_965.0  \
0 -1.587793       1.394285      -0.539864  ...     -0.033004     -0.022709   
1 -1.813807       1.427786      -0.303917  ...     -0.033004     -0.022709   
2 -1.813807       1.427786      -0.303917  ...     -0.033004     -0.022709   
3 -1.813807       1.427786      -0.303917  ...     -0.033004     -0.022709   
4 -1.587793       1.394285      -0.539864  ...     -0.033004     -0.022709   

   SiteID_968.0  SiteID_974.0  SiteID_1002.0  day_of_week_0.0  \
0     -0.031673

In [51]:
#print(Xtrain_KNN_encoded.head())
#print(Xtest_KNN_encoded.head()) 


#print(Xtrain_KNN_Killian.head())
#print(Xtest_KNN_Killian.head()) 

# print(Xtrain_KNN.shape) 
# print(Xtest_KNN.shape)  

# print(ytrain.shape)
# print(ytest.shape)

# print(Xtrain_1.shape)
# print(Xtest_1.shape)

# print(ytrain.shape)
# print(ytest.shape)

print(Xtrain_KNN_Killian.shape)
print(Xtest_KNN_Killian.shape)

print(Xtrain_KNN_encoded.shape) 
print(Xtest_KNN_encoded.shape)

(34922, 84)
(8271, 84)
(34923, 901)
(8272, 901)


Linear Regression

In [16]:
linear_model = LinearRegression()

# You can still use TimeSeriesSplit for cross-validation to evaluate model performance
tscv = TimeSeriesSplit(n_splits=5)

# Evaluate the model using cross-validation
scores = cross_val_score(linear_model, Xtrain_1, ytrain, cv=tscv, scoring='neg_mean_squared_error')

# Print the average of the scores (neg_mean_squared_error)
print("Average MSE:", np.mean(scores))

Average MSE: -1.5702208364533915e+25


In [17]:
# Initialize the Linear Regression model
linear_model = LinearRegression()

# Fit the model on the training data
linear_model.fit(Xtrain_1, ytrain)

# Predict on the test data
ypred_linear = linear_model.predict(Xtest_1)

# Calculate the mean squared error
mse = mean_squared_error(ytest, ypred_linear)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 36.2921732516979


Linear Regression with KNN imputed data 

In [40]:
linear_model = LinearRegression()

# You can still use TimeSeriesSplit for cross-validation to evaluate model performance
tscv = TimeSeriesSplit(n_splits=5)

# Evaluate the model using cross-validation
scores = cross_val_score(linear_model, Xtrain_KNN, ytrain, cv=tscv, scoring='neg_mean_squared_error')

# Print the average of the scores (neg_mean_squared_error)
print("Average MSE:", np.mean(scores))

Average MSE: -3.498884230876506e+25


In [41]:
# Initialize the Linear Regression model
linear_model = LinearRegression()

# Fit the model on the training data
linear_model.fit(Xtrain_KNN, ytrain)

# Predict on the test data
ypred_linear = linear_model.predict(Xtest_KNN)

# Calculate the mean squared error
mse = mean_squared_error(ytest, ypred_linear)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 35.76472089608074


Ridge 

In [18]:
# Initialize Ridge Regression
ridge = Ridge()
tscv = TimeSeriesSplit(n_splits=5)
# Define parameter grid
param_grid = {
    'alpha': np.logspace(-4, 4, 100)  # This creates 20 logarithmically spaced values between 10^-4 and 10^4.
}


In [19]:
# Setup GridSearchCV
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [20]:
# Fit GridSearchCV to find the best model
grid_search.fit(Xtrain_1, ytrain)

# Output the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best score (negative MSE):", grid_search.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'alpha': 6892.61210434971}
Best score (negative MSE): -52.04375644919869


In [21]:
# Initialize the Ridge Regression model with the best alpha
ridge_model = Ridge(alpha= 6892.61210434971)

# Fit the model on the training data
ridge_model.fit(Xtrain_1, ytrain) 

Ridge(alpha=6892.61210434971)

In [22]:
ypred_ridge = ridge_model.predict(Xtest_1)  # Predict on the test data
mse = mean_squared_error(ytest, ypred_ridge)  # Calculate the mean squared error
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 28.355387843423312


Ridge with KNN

In [42]:
# Fit GridSearchCV to find the best model
grid_search.fit(Xtrain_KNN, ytrain)

# Output the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best score (negative MSE):", grid_search.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
# Initialize the Ridge Regression model with the best alpha
ridge_model = Ridge(alpha= )

# Fit the model on the training data
ridge_model.fit(Xtrain_KNN, ytrain) 

In [ ]:
ypred_ridge = ridge_model.predict(Xtest_KNN)  # Predict on the test data
mse = mean_squared_error(ytest, ypred_ridge)  # Calculate the mean squared error
print(f"Mean Squared Error: {mse}")

Lasso 

In [23]:
# Define parameter grid
lasso = Lasso()
param_grid = {
    'alpha': np.logspace(-4, 4, 100)  # Creates 100 logarithmically spaced values between 10^-4 and 10^4.
}

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=tscv, 
                           scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
# Assuming Xtrain_1 and ytrain are already defined and are your feature and target datasets respectively
# Fit GridSearchCV to find the best model
grid_search.fit(Xtrain_1, ytrain)

# Output the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best score (negative MSE):", grid_search.best_score_)

In [24]:
# Initialize the lasso Regression model with the best alpha
lasso_model = Lasso(alpha= 0.06734150657750829)

# Fit the model on the training data
lasso_model.fit(Xtrain_1, ytrain) 

Lasso(alpha=0.06734150657750829)

In [25]:
ypred_lasso = lasso_model.predict(Xtest_1)  # Predict on the test data
mse = mean_squared_error(ytest, ypred_lasso)  # Calculate the mean squared error
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 27.566745580198322


In [ ]:
# # Generate the confusion matrix
# # Define a threshold for classification
# threshold = 0.5 
# ypred_lasso_classification = (ypred_lasso >= threshold).astype(int)


In [28]:
# # Generate the confusion matrix
# conf_matrix = confusion_matrix(ytest, ypred_lasso_classification)
# conf_matrix

print(ypred_linear)
print(ypred_ridge)
print(ypred_lasso)

[ 9.1189152  14.0744816   9.50856363 ...  4.84582141  6.37194446
  6.53356555]
[ 9.8217414  13.94405013  9.44328738 ...  4.83296613  6.36363501
  6.1354454 ]
[ 9.65197409 11.57363711  8.41336532 ...  5.1809495   6.22731267
  6.95238962]


In [26]:
 from sklearn.metrics import accuracy_score

# # y_pred_ridge, y_pred_lasso, and y_pred_log are the predictions from Ridge, Lasso, and Logistic Regression models, respectively

# #accuracy_ridge = accuracy_score(ytest, ypred_ridge)
accuracy_lasso = accuracy_score(ytest, ypred_lasso)
# #accuracy_linear = accuracy_score(ytest, ypred_linear)

ValueError: continuous is not supported

In [30]:
def get_model_cv_score(model, X, y, scoring='neg_mean_squared_error'):
    scores = cross_val_score(model, X, y, cv=5, scoring=scoring)
    return np.mean(scores)

linear_score = get_model_cv_score(linear_model,Xtrain_1, ytrain)
ridge_score = get_model_cv_score(ridge_model, Xtrain_1, ytrain)
lasso_score = get_model_cv_score(lasso_model, Xtrain_1, ytrain)

print(f"Linear Regression Score: {linear_score}")
print(f"Ridge Regression Score: {ridge_score}")
print(f"Lasso Regression Score: {lasso_score}")


Linear Regression Score: -1.1157565394021618e+26
Ridge Regression Score: -49.38105120954005
Lasso Regression Score: -49.75704782817994


Kernel Ridge Regression 

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load your dataset
#f#ull_data = pd.read_csv('path_to_your_data.csv')

# Preprocess Data
# Assuming 'date' is the datetime column and 'PM25_ugm3' is the target variable
full_data['date'] = pd.to_datetime(full_data['date'])
full_data.sort_values('date', inplace=True)

# Split features and target
X = full_data.drop('PM25_ugm3', axis=1)
y = full_data['PM25_ugm3'].values

# Identify categorical and numerical columns
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Create preprocessing pipelines
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Setup the Ridge Regression model within a pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', Ridge())])

# Setup Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Model training and cross-validation
mse_scores = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)
    print(f'MSE: {mse}')

# Output average MSE
print(f'Average MSE: {np.mean(mse_scores)}')


MSE: 71.44571516666859
MSE: 43.99941896659314
MSE: 51.826019430590556
MSE: 33.886507612469856
MSE: 25.638645251445833
Average MSE: 45.3592612855536
